TODO: HANDLE PAGINATION

2. Select the last 4 month range
3. Get all the speech urls from the returned result
4. Fetch each page and extract speech body
5. Analyze each speech for H and D using dictionaries.

In [152]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
import os

In [153]:
FED_BASE_URL = "https://www.federalreserve.gov"
FED_SPEECHES_URL = "https://www.federalreserve.gov/newsevents/speeches.htm"

In [154]:
print("Instanciating Chrome web driver")

HOME_DIR = "/Users/dennisfenchenko/"

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

Instanciating Chrome web driver


In [155]:
START_DATE = "06/20/2022"
END_DATE = "10/20/2022"

def get_fed_speeches(start_date, end_date, pagination=1):
    START_DATE_XPATH = "//*[@id=\"content\"]/div[2]/div/div[1]/form/div[2]/div/div[1]/input"
    END_DATE_XPATH = "//*[@id=\"content\"]/div[2]/div/div[1]/form/div[2]/div/div[2]/input"
    SUBMIT_BUTTON_XPATH = "//*[@id=\"content\"]/div[2]/div/div[1]/form/div[5]/a"
    driver.get(FED_SPEECHES_URL)
    start_date_input_box = driver.find_element("xpath", START_DATE_XPATH)
    end_date_input_box = driver.find_element("xpath", END_DATE_XPATH)
    start_date_input_box.clear()
    start_date_input_box.send_keys(start_date)
    end_date_input_box.clear()
    end_date_input_box.send_keys(end_date)
    submit_button = driver.find_element("xpath", SUBMIT_BUTTON_XPATH)
    submit_button.click()
    return driver.page_source

In [156]:
doc_body = get_fed_speeches(START_DATE, END_DATE)

In [157]:
def get_fed_speech_links(all_speeches):
    soup = BeautifulSoup(all_speeches, 'html.parser')
    fed_page_links = soup.find_all("a", href=True)
    fed_speech_transcript_links = [link['href'] for \
            link in fed_page_links if "/speech/" in link['href']]
    return fed_speech_transcript_links

In [158]:
import re

def get_fed_speech_transcript_links(fed_speech_links):
    DATE_LENGTH = 8
    fed_speech_transcript_links = []
    for link in fed_speech_links:
        date = re.findall('[0-9]+', link)
        if len(date) != 0:
            date = date[0]
        if len(date) == DATE_LENGTH:
            fed_speech_transcript_links.append(link)
    return fed_speech_transcript_links

In [159]:
fed_page_links = get_fed_speech_links(doc_body)
fed_page_transcript_links = get_fed_speech_transcript_links(fed_page_links)

In [161]:
def filter_duplicate_links(speech_links):
    uniq_links = set(speech_links)
    return list(uniq_links)

In [162]:
speech_links = filter_duplicate_links(fed_page_transcript_links)

In [168]:
def get_fed_speeches(fed_speech_links):
    fed_speeches = []
    for link in fed_speech_links:
        fed_speeches.append(requests.get(FED_BASE_URL + link).text)
    return fed_speeches

In [169]:
fed_speeches = get_fed_speeches(speech_links)

In [191]:
def get_speech(fed_speech_links, fed_speeches_source):
    fed_speech_content = {}
    num_speeches = len(fed_speech_links)
    for speech_idx in range(num_speeches):
        date = re.findall('[0-9]+', fed_speech_links[speech_idx])[0]
        soup = BeautifulSoup(fed_speeches_source[speech_idx], "html.parser")
        speech_text = ""
        for paragraph in soup.find_all('p'):
            speech_text += paragraph.get_text()
        fed_speech_content[date] = speech_text
    return fed_speech_content

In [192]:
speeches = get_speech(speech_links, fed_speeches)

In [193]:
speeches

{'20221006': '\nThe Federal Reserve, the central bank of the United States, provides\r\n          the nation with a safe, flexible, and stable monetary and financial\r\n          system.\n\nFederal Open Market Committee\n\nMonetary Policy Principles and Practice\n\nPolicy Implementation\n\nReports\n\nReview of Monetary Policy Strategy, Tools, and\r\n                    Communications\n\nInstitution Supervision\n\nReports\n\nReporting Forms\n\nSupervision & Regulation Letters\n\nBanking Applications & Legal Developments\n\nRegulatory Resources\n\nBanking & Data Structure\n\nFinancial Stability Assessments \n\nFinancial Stability Coordination & Actions\n\nReports\n\nRegulations & Statutes\n\nPayment Policies\n\nReserve Bank Payment Services & Data\n\nFinancial Market Utilities & Infrastructures\n\nResearch, Committees, and Forums\n\nWorking Papers and Notes\n\nData, Models and Tools\n\nBank Assets and Liabilities\n\nBank Structure Data\n\nBusiness Finance\n\nDealer Financing Terms\n\nExc